## The purpose of this project to categories theme of news article
(do it in fastcampus class with Park)

### 1. explain

#### variable
- IV : the contents of news article
- DV : the number of categories
    - detail

|num|name|
|---|---|
|100|polictic|
|101|economic|
|102|social|
|103|life/culture|
|104|world|
|105|IT/science|


#### Stategy
![process](images/article_preocess.png)

### 2. develop

requirements

In [94]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, KFold

prepare datasets

In [84]:
article_df = pd.read_csv("datas/article.csv")
len(article_df)

31088

In [85]:
article_df.head(5)

,title,link,content,category
0,"바른미래, '유승민·오신환·권은희·유의동' 중징계에 ""효력없어""반론도",https://news.naver.com/main/read.nhn?mode=LSD&...,"오신환 ""손학규 막장 정치에 환멸 느낀다. 원내대표 끝까지 수행 할 것"" 손학...",100
1,"주말동안 책읽은 文대통령, SNS서 도올 김용옥 책 추천",https://news.naver.com/main/read.nhn?mode=LSD&...,문재인 대통령이 휴일인 1일 “금요일 하루 연가를 낸 덕분에 주말 동안 책 세 권을...,100
2,"백원우 휘하 '하명수사' 의혹 특감반원 사망에 靑 ""입장 낼 사안 아냐""",https://news.naver.com/main/read.nhn?mode=LSD&...,청와대 전경. 연합뉴스 백원우 전 청와대 민정비서관(현 민주연구원 부원장) ...,100
3,"오신환 ""손학규 막장정치에 환멸…원내대표직 계속 수행""",https://news.naver.com/main/read.nhn?mode=LSD&...,"[머니투데이 박종진 기자] [[the300]당 윤리위, 유승민 등에 당원권 정지 ...",100
4,"[저널리즘 토크쇼 J] 언론의 ‘지소미아’보도, 누구의 시각인가",https://news.naver.com/main/read.nhn?mode=LSD&...,"[정세진] 여러분, 안녕하십니까? <저널리즘 토크쇼 J>입니다. 오늘 함께하실 분들...",100


In [87]:
article_df["content"] = article_df["content"].apply(lambda x : str(x))
X = article_df.content.tolist()
y = article_df.category.tolist()

In [89]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(27979, 3109, 27979, 3109)

make pipeline

In [91]:
clf1 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

clf2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

training

In [78]:
model1 = clf1.fit(X_train, y_train)
model2 = clf2.fit(X_train, y_train)

test

In [97]:
for i, model in enumerate([model1, model2]):
    y_pred = model.predict(X_test)
    scores = accuracy_score(y_test, y_pred)
    print(("Model{0:d}: test_accuracy: {1:.3f}").format(i + 1, scores))

Model1: test_accuracy: 0.834
Model2: test_accuracy: 0.837


In [98]:
for i, model in enumerate([model1, model2]):
    scores = cross_val_score(model, X, y, cv=5)
    print(("Model{0:d}: Mean score: {1:.3f}").format(i + 1, np.mean(scores)))

Model1: Mean score: 0.793
Model2: Mean score: 0.795


### 3. Model use

In [102]:
categories = {
    100: "politic",
    101: "economic",
    102: "social",
    103: "life/culture",
    104: "world",
    105: "IT/science",
}

create news article

In [103]:
contents = [
    "네이버와 카카오는 드론 기술 발전에 주력",
    "요즘 환율 주가 예측이 불가",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
]

In [104]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,요즘 환율 주가 예측이 불가,101
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",100


show predcit result

In [107]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model2.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/science,0.94
1,요즘 환율 주가 예측이 불가,101,economic,0.77
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",100,politic,0.58


In [108]:
# 모델 저장

In [109]:
pickle.dump(model, open("clf.pkl", "wb"))